In [ ]:
from visualization_fct import *
# from bokeh.plotting import output_file, show, save
from bokeh.io import output_notebook
output_notebook()
from bokeh.plotting import show  # output_file, save

from bokeh.resources import CDN
from bokeh.embed import file_html

from pystan import stan

import matplotlib.pyplot as plt  # , mpld3

without_CA = False

data = pd.read_csv("../asm_data/asm_data_for_ml.txt", sep='\t')
del data['MJD']
# del data['error']
# del data['errorA']
# del data['errorB']
# del data['errorC']
data['rateCA'] = data.rateC / data.rateA
data_thr = mask(data, 'orbit')  # rm too large values except for 'orbit'

w = np.concatenate([np.array(data_thr[name]).reshape(-1, 1)
                    for name in ['error', 'errorA', 'errorB', 'errorC']],
                   axis=1)
w = np.exp(-w.mean(axis=1))

del data_thr['error']
del data_thr['errorA']
del data_thr['errorB']
del data_thr['errorC']


np.random.seed(0)

if without_CA:
    X = np.c_[data_thr.orbit, data_thr.rate, data_thr.rateA, data_thr.rateB,
              data_thr.rateC]
else:
    X = np.c_[data_thr.orbit, data_thr.rate, data_thr.rateA, data_thr.rateB,
              data_thr.rateC, data_thr.rateCA]

gmm = GeneralMixtureModel(MultivariateGaussianDistribution, n_components=3)
gmm.fit(X, weights=w)



In [ ]:
preds = gmm.predict(X)
probs = gmm.predict_proba(X)

data_thr['preds'] = pd.Series(preds).astype("category")

color_key = ["red", "blue", "yellow", "grey", "black", "purple", "pink",
             "brown", "green", "orange"]  # Spectral9
color_key = color_key[:len(set(preds))+1]

covs = np.array([np.array(gmm.distributions[m].parameters[1])
                 for m in range(len(gmm.distributions))])
means = np.array([np.array(gmm.distributions[m].parameters[0])
                  for m in range(len(gmm.distributions))])

In [ ]:
print w.shape
print X.shape
data['error'].reshape(-1, 1).shape

In [ ]:
# single plot rateCA vs rate with predicted classes and ellipses:
if without_CA:
    covs_xy = None
    means_xy = None
else:
    x = 5
    y = 1
    covs_xy = [covs[j][[x, y]][:, [x, y]] for j in range(len(covs))]
    means_xy = [means[j][[x, y]] for j in range(len(covs))]

single_plot = bokeh_datashader_plot(data_thr, covs=covs_xy, means=means_xy,
                                    x_name='rateCA',
                                    y_name='rate',
                                    plot_width=900, plot_height=300,
                                    pixel_width=3000, pixel_height=1000,
                                    spread=True, color_key=color_key)
show(single_plot)